In [1]:
from config import *
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import roc_auc_score
from main_package.utils import data_path_to_abs_path

In [2]:
train_data = "piech/train_original"
test_data = "piech/test_original"

def read_sessions(filename):
    f = open(data_path_to_abs_path(filename))
    sessions = []
    while True:
        line = f.readline()
        if line == "":
            break
        num_records = int(line.strip())
        skills = np.array(f.readline().strip().split(",")[:-1], dtype=int)
        answers = np.array(f.readline().strip().split(",")[:-1], dtype=int)
        if num_records > 1:
            sessions.append((skills, answers))
    f.close()
    return sessions

sessions_train = read_sessions(train_data)
sessions_test = read_sessions(test_data)

In [3]:
#this cell is for making the test dataset available in other format
def from_sessions_to_df(sessions, name):
    skills, answers = sessions[0]
    student_id = 0
    students = np.zeros((len(skills)), dtype=int)
    for session in sessions[1:]:
        student_id += 1
        students = np.append(students, [student_id]*len(session[0]))
        skills = np.append(skills, session[0])
        answers = np.append(answers, session[1])
    
    df = pd.DataFrame({"user_id": students, "skill_id": skills, "correct": answers})
    #df.to_csv(name, index=False)
    return df

df_test_piech = from_sessions_to_df(sessions_test, "piech_test_df_format.csv")
df_train_piech = from_sessions_to_df(sessions_train, "piech_train_df_format.csv")

In [4]:
print(f"train + test dataset length: {len(df_train_piech)+len(df_test_piech)}")
print(f"training dataset length: {len(df_train_piech)}")
print(f"testing dataset length: {len(df_test_piech)}")
print(f"test ratio: {len(df_test_piech) / (len(df_train_piech)+len(df_test_piech))}")
df_train_piech.to_csv("piech_all_df_format.csv", index=False)
df_test_piech.to_csv("piech_all_df_format.csv", index=False, header=False, mode='a')
df_all = pd.read_csv("piech_all_df_format.csv")
df_all.to_csv("piech_all_df_format.csv", index=True, index_label='order_id')

train + test dataset length: 525425
training dataset length: 407880
testing dataset length: 117545
test ratio: 0.223714136175477


In [9]:
print(df_train_piech['correct'].value_counts(normalize=True))
print(df_test_piech['correct'].value_counts(normalize=True))
print(df_all['correct'].value_counts(normalize=True))

1    0.676954
0    0.323046
Name: correct, dtype: float64
1    0.688451
0    0.311549
Name: correct, dtype: float64
1    0.679526
0    0.320474
Name: correct, dtype: float64


In [7]:
def bkt_single_skill(session, p_L0: float, p_G: float, p_S: float, p_T: float):
    skill, right = session
    assert len(skill) == len(right)
    n = len(skill)
    current_skill = -1
    assert current_skill not in skill
    p_prior, p_prior_cond, p_posterior = 0, 0, 0 #PLn-1, PLn-1|answer, PLn
    posteriors = np.empty((n), dtype=np.float64)
    priors = np.empty((n), dtype=np.float64)
    squared_sum_residuals = 0
    for i in range(n):
        if skill[i] == current_skill:
            p_prior = posteriors[i-1]
        else:
            p_prior = p_L0
            current_skill = skill[i]
        priors[i] = p_prior

        if right[i]:
            p_did_not_slip_and_knew = (1 - p_S) * p_prior
            p_guessed_and_did_not_know = p_G * (1 - p_prior)
            p_prior_cond = p_did_not_slip_and_knew / (p_did_not_slip_and_knew + p_guessed_and_did_not_know)
        else:
            p_slipped_and_knew = p_S * p_prior
            p_did_not_guess_and_did_not_know = (1 - p_G) * (1 - p_prior)
            p_prior_cond = p_slipped_and_knew / (p_slipped_and_knew + p_did_not_guess_and_did_not_know)
        
        p_posterior = p_prior_cond + p_T * (1 - p_prior_cond)
        posteriors[i] = p_posterior

        likelihood_correct = p_prior * (1 - p_S) + (1 - p_prior) * p_G
        squared_sum_residuals += (right[i] - likelihood_correct)**2
    return priors, squared_sum_residuals

In [8]:
def bkt_minimize(params, sessions):
    p_L0, p_G, p_S, p_T = params
    #print(f"p_L0: {p_L0}, p_G: {p_G}, p_S: {p_S}, p_T: {p_T}")
    result = 0
    for session in sessions:
        _, ssr = bkt_single_skill(session, p_L0, p_G, p_S, p_T)
        result += ssr
    #print(f"squared_sum_residuals: {result}")
    return result

In [12]:
initial_guess = [0.4, 0.2, 0.08, 0.1] #p_L0, p_G, p_S, p_T
dx = 0.0001
bounds = [(dx, 1-dx), (dx, 0.3), (dx, 0.1), (dx, 1-dx)]
result = minimize(bkt_minimize, initial_guess, args=sessions_train, bounds=bounds, method='L-BFGS-B')
#method L-BFGS-B is the default for bounded minimization
print(f"p_L0 = {result.x[0]}\np_G = {result.x[1]}\np_S = {result.x[2]}\np_T = {result.x[3]}")
print(result)

p_L0 = 0.5504063365919529
p_G = 0.3
p_S = 0.0999999975569658
p_T = 0.06302321901515896
      fun: 76382.33392676056
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 1.75667809e+02, -2.91853747e+06, -3.08436575e+04, -2.79699452e+02])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 310
      nit: 15
     njev: 62
   status: 0
  success: True
        x: array([0.55040634, 0.3       , 0.1       , 0.06302322])


EVALUATION

In [9]:
"""
model fit for test :) - hups
p_L0 = 0.5641198789451226
p_G = 0.3
p_S= 0.09999998796996834
p_T= 0.056647248212908334

newer model fit for train
p_L0 = 0.5504063365919529
p_G = 0.3
p_S = 0.0999999975569658
p_T = 0.06302321901515896

original model fit for train
p_L0 = 0.5497157313235409
p_G = 0.3
p_S = 0.0999999919596138
p_T = 0.0631258533486059
"""

# model fit for train
p_L0 = 0.5504063365919529
p_G = 0.3
p_S = 0.0999999975569658
p_T = 0.06302321901515896

In [10]:
prediction, ssr_total = bkt_single_skill(sessions_test[0], p_L0, p_G, p_S, p_T)
correct = np.copy(sessions_test[0][1])
for session in sessions_test[1:]:
    estimate, ssr = bkt_single_skill(session, p_L0, p_G, p_S, p_T)
    correct = np.append(correct, session[1])
    prediction = np.append(prediction, estimate)
    ssr_total += ssr
print(f"number of records: {len(prediction)}")
print(f"squared sum of residuals: {ssr_total}")
print(f"auc: {roc_auc_score(correct, prediction)}")


number of records: 117545
squared sum of residuals: 21684.46449641108
auc: 0.7295786468978473
